In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('Surgical-deepnet.csv')

In [3]:
data

,bmi,Age,asa_status,baseline_cancer,baseline_charlson,baseline_cvd,baseline_dementia,baseline_diabetes,baseline_digestive,baseline_osteoart,...,complication_rsi,dow,gender,hour,month,moonphase,mort30,mortality_rsi,race,complication
0,19.31,59.2,1,1,0,0,0,0,0,0,...,-0.57,3,0,7.63,6,1,0,-0.43,1,0
1,18.73,59.1,0,0,0,0,0,0,0,0,...,0.21,0,0,12.93,0,1,0,-0.41,1,0
2,21.85,59.0,0,0,0,0,0,0,0,0,...,0.00,2,0,7.68,5,3,0,0.08,1,0
3,18.49,59.0,1,0,1,0,0,1,1,0,...,-0.65,2,1,7.58,4,3,0,-0.32,1,0
4,19.70,59.0,1,0,0,0,0,0,0,0,...,0.00,0,0,7.88,11,0,0,0.00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14630,18.79,14.1,1,0,1,0,0,0,0,0,...,-0.54,1,0,7.78,0,1,0,-0.16,1,1
14631,19.65,12.6,0,0,0,0,0,0,0,0,...,-1.42,4,0,8.40,6,1,0,-0.77,1,1
14632,14.84,12.6,1,0,0,0,0,0,0,0,...,0.65,0,0,13.25,3,3,0,0.99,1,1
14633,17.75,8.9,0,0,1,0,0,0,1,0,...,-0.50,0,1,8.30,5,0,0,0.17,1,1


In [4]:
data.dtypes

bmi                    float64
Age                    float64
asa_status               int64
baseline_cancer          int64
baseline_charlson        int64
baseline_cvd             int64
baseline_dementia        int64
baseline_diabetes        int64
baseline_digestive       int64
baseline_osteoart        int64
baseline_psych           int64
baseline_pulmonary       int64
ahrq_ccs                 int64
ccsComplicationRate    float64
ccsMort30Rate          float64
complication_rsi       float64
dow                      int64
gender                   int64
hour                   float64
month                    int64
moonphase                int64
mort30                   int64
mortality_rsi          float64
race                     int64
complication             int64
dtype: object

In [5]:
data = data.astype('float64')

In [6]:
data.dtypes

bmi                    float64
Age                    float64
asa_status             float64
baseline_cancer        float64
baseline_charlson      float64
baseline_cvd           float64
baseline_dementia      float64
baseline_diabetes      float64
baseline_digestive     float64
baseline_osteoart      float64
baseline_psych         float64
baseline_pulmonary     float64
ahrq_ccs               float64
ccsComplicationRate    float64
ccsMort30Rate          float64
complication_rsi       float64
dow                    float64
gender                 float64
hour                   float64
month                  float64
moonphase              float64
mort30                 float64
mortality_rsi          float64
race                   float64
complication           float64
dtype: object

In [7]:
data.isnull().sum()

bmi                    0
Age                    0
asa_status             0
baseline_cancer        0
baseline_charlson      0
baseline_cvd           0
baseline_dementia      0
baseline_diabetes      0
baseline_digestive     0
baseline_osteoart      0
baseline_psych         0
baseline_pulmonary     0
ahrq_ccs               0
ccsComplicationRate    0
ccsMort30Rate          0
complication_rsi       0
dow                    0
gender                 0
hour                   0
month                  0
moonphase              0
mort30                 0
mortality_rsi          0
race                   0
complication           0
dtype: int64

In [10]:
print(data.duplicated())
data.drop_duplicates(inplace = True)

0        False
1        False
2        False
3        False
4        False
         ...  
14630    False
14631    False
14632    False
14633    False
14634    False
Length: 11733, dtype: bool


In [11]:
data.columns = data.columns.str.lower()

In [13]:
data

,bmi,age,asa_status,baseline_cancer,baseline_charlson,baseline_cvd,baseline_dementia,baseline_diabetes,baseline_digestive,baseline_osteoart,...,complication_rsi,dow,gender,hour,month,moonphase,mort30,mortality_rsi,race,complication
0,19.31,59.2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.57,3.0,0.0,7.63,6.0,1.0,0.0,-0.43,1.0,0.0
1,18.73,59.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.21,0.0,0.0,12.93,0.0,1.0,0.0,-0.41,1.0,0.0
2,21.85,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,2.0,0.0,7.68,5.0,3.0,0.0,0.08,1.0,0.0
3,18.49,59.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,-0.65,2.0,1.0,7.58,4.0,3.0,0.0,-0.32,1.0,0.0
4,19.70,59.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,7.88,11.0,0.0,0.0,0.00,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14630,18.79,14.1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.54,1.0,0.0,7.78,0.0,1.0,0.0,-0.16,1.0,1.0
14631,19.65,12.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.42,4.0,0.0,8.40,6.0,1.0,0.0,-0.77,1.0,1.0
14632,14.84,12.6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.65,0.0,0.0,13.25,3.0,3.0,0.0,0.99,1.0,1.0
14633,17.75,8.9,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-0.50,0.0,1.0,8.30,5.0,0.0,0.0,0.17,1.0,1.0


In [14]:
x = data.drop(labels = 'complication' , axis = 1)
y = data[['complication']]

In [15]:
train_data , test_data , train_labels , test_labels = train_test_split(x, y, train_size = 0.7, test_size = 0.3, random_state = 42)

In [16]:
sc = StandardScaler()
train_data = sc.fit_transform(train_data)
test_data = sc.transform(test_data)

In [17]:
rem_train_data, val_data, rem_train_labels, val_labels = train_test_split(train_data, train_labels, 
                                                                          test_size = 0.2, random_state = 42)

In [32]:
print(rem_train_data.shape)
print(val_data.shape)
print(test_data.shape)

(6570, 24)
(1643, 24)
(3520, 24)


array([[-1.0476303 ,  1.10212478,  0.805184  , ..., -0.0699583 ,
         0.32569059,  0.24520366],
       [-0.53299667, -0.03172128, -0.96157427, ..., -0.0699583 ,
         0.07556517,  0.24520366],
       [ 1.95702103,  0.30696001,  0.805184  , ..., -0.0699583 ,
         0.05703736, -2.23186599],
       ...,
       [-0.89368967,  0.225971  , -0.96157427, ..., -0.0699583 ,
        -0.25793539,  0.24520366],
       [-0.72514154,  1.07267423, -0.96157427, ..., -0.0699583 ,
        -1.47150688,  0.24520366],
       [-0.06780384, -0.03172128,  0.805184  , ..., -0.0699583 ,
        -1.69384059,  0.24520366]])

In [41]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape = (rem_train_data.shape[1],)))
model.add(layers.Dropout(0))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dropout(0))
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [44]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.05),
             loss = 'binary_crossentropy', 
             metrics = ['accuracy'])

In [45]:
Result = model.fit(rem_train_data, rem_train_labels, epochs = 100, batch_size = 64, validation_data = (val_data , val_labels))

Epoch 1/100
103/103 [==============================] - 1s 6ms/step - loss: 0.5721 - accuracy: 0.6962 - val_loss: 0.5323 - val_accuracy: 0.7596
Epoch 2/100
103/103 [==============================] - 0s 2ms/step - loss: 0.5253 - accuracy: 0.7501 - val_loss: 0.5330 - val_accuracy: 0.7596
Epoch 3/100
103/103 [==============================] - 0s 2ms/step - loss: 0.5252 - accuracy: 0.7513 - val_loss: 0.5229 - val_accuracy: 0.7559
Epoch 4/100
103/103 [==============================] - 0s 2ms/step - loss: 0.5266 - accuracy: 0.7514 - val_loss: 0.5179 - val_accuracy: 0.7559
Epoch 5/100
103/103 [==============================] - 0s 2ms/step - loss: 0.5233 - accuracy: 0.7482 - val_loss: 0.5488 - val_accuracy: 0.7553
Epoch 6/100
103/103 [==============================] - 0s 2ms/step - loss: 0.5174 - accuracy: 0.7537 - val_loss: 0.5262 - val_accuracy: 0.7578
Epoch 7/100
103/103 [==============================] - 0s 2ms/step - loss: 0.5201 - accuracy: 0.7556 - val_loss: 0.5289 - val_accuracy: 0.7608

Epoch 58/100
103/103 [==============================] - 0s 2ms/step - loss: 0.4885 - accuracy: 0.7604 - val_loss: 0.5356 - val_accuracy: 0.7669
Epoch 59/100
103/103 [==============================] - 0s 2ms/step - loss: 0.4787 - accuracy: 0.7763 - val_loss: 0.5242 - val_accuracy: 0.7681
Epoch 60/100
103/103 [==============================] - 0s 2ms/step - loss: 0.4784 - accuracy: 0.7769 - val_loss: 0.5334 - val_accuracy: 0.7687
Epoch 61/100
103/103 [==============================] - 0s 2ms/step - loss: 0.4820 - accuracy: 0.7700 - val_loss: 0.5519 - val_accuracy: 0.7638
Epoch 62/100
103/103 [==============================] - 0s 2ms/step - loss: 0.4848 - accuracy: 0.7705 - val_loss: 0.5229 - val_accuracy: 0.7596
Epoch 63/100
103/103 [==============================] - 0s 2ms/step - loss: 0.4827 - accuracy: 0.7725 - val_loss: 0.5522 - val_accuracy: 0.7620
Epoch 64/100
103/103 [==============================] - 0s 2ms/step - loss: 0.4785 - accuracy: 0.7743 - val_loss: 0.5413 - val_accuracy:

In [40]:
Evaluation = model.evaluate(test_data, test_labels)
print(Evaluation) 

110/110 [==============================] - 0s 2ms/step - loss: 0.5992 - accuracy: 0.7233
[0.5992357134819031, 0.7232954502105713]
